In [4]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

### Question 2: 

**FHV October 2019**

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 1MB
- 6MB
- 25MB
- 87MB


In [5]:
spark = SparkSession.builder.master("local[*]").appName('hw').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 13:47:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/28 13:48:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.read.option("header", "true").csv('fhv_data/fhv_tripdata_2019-10.csv.gz')

In [5]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
schema = types.StructType(
    [types.StructField('dispatching_base_num', types.StringType(), True), 
     types.StructField('pickup_datetime', types.TimestampType(), True), 
     types.StructField('dropOff_datetime', types.TimestampType(), True), 
     types.StructField('PUlocationID', types.IntegerType(), True), 
     types.StructField('DOlocationID', types.IntegerType(), True), 
     types.StructField('SR_Flag', types.StringType(), True), 
     types.StructField('Affiliated_base_number', types.StringType(), True)]
)

In [9]:
df = spark.read.option("header", "true").schema(schema).csv('fhv_data/fhv_tripdata_2019-10.csv.gz')

In [10]:
df.show(10)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [11]:
df.repartition(6).write.parquet('fhv_data/2019/')

AnalysisException: path file:/home/taras/DE-ZoomCamp-2024/Week-5/fhv_data/2019 already exists.

In [21]:
!ls -lah fhv_data/2019

total 39M
drwxr-xr-x 2 taras taras 4.0K Feb 28 12:27 .
drwxrwxr-x 3 taras taras 4.0K Feb 28 12:27 ..
-rw-r--r-- 1 taras taras    8 Feb 28 12:27 ._SUCCESS.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00000-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00001-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00002-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00003-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00004-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras  51K Feb 28 12:27 .part-00005-127a01a6-d78a-47d9-901f-bf28088f48b5-c000.snappy.parquet.crc
-rw-r--r-- 1 taras taras    0 Feb 28 12:27 _SUCCESS
-rw-r--r-- 1 taras taras 6.4M Feb 28 12:27 part-00000-127a01a6-d78a-47d9-901f-bf28088f48b5-c00

### Question 3: 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.


In [12]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)).\
    filter("pickup_date = '2019-10-15'").count()

62610

### Question 4: 

**Longest trip for each day** 

What is the length of the longest trip in the dataset in hours?

In [14]:
df.withColumn('duration', (df.dropOff_datetime.cast('long')
              -df.pickup_datetime.cast('long'))/3600).\
              withColumn('pickup_date', F.to_date(df.pickup_datetime)).\
              groupBy('pickup_date').max('duration').orderBy('max(duration)', ascending=False).limit(5).show()

+-----------+-----------------+
|pickup_date|    max(duration)|
+-----------+-----------------+
| 2019-10-28|         631152.5|
| 2019-10-11|         631152.5|
| 2019-10-31|87672.44083333333|
| 2019-10-01|70128.02805555555|
| 2019-10-17|           8794.0|
+-----------+-----------------+

